# 你放我抓玩法  Grab play

### 导入头文件 Import head file

In [ ]:
#!/usr/bin/env python
# coding: utf-8
import cv2 as cv
import threading
from time import sleep
from dofbot_config import *
import ipywidgets as widgets
from IPython.display import display
from color_sorting import color_sorting

### 初始化机械臂位置 Initialize DOFBOT position

In [ ]:
import Arm_Lib
Arm = Arm_Lib.Arm_Device()
joints_0 = [80, 130, 0, 0, 90, 30]
Arm.Arm_serial_servo_write6_array(joints_0, 1000)

### 创建实例,初始化参数 Create the instance and initialize the parameters

In [ ]:
sorting = color_sorting()
model = 'General'
color_hsv  = {"red"   : ((0, 43, 46), (10, 255, 255)),
              "green" : ((35, 43, 46), (77, 255, 255)),
              "blue"  : ((100, 43, 46), (124, 255, 255)),
              "yellow": ((26, 43, 46), (34, 255, 255))}
# HSV参数路径  HSV Parameter path
HSV_path="/home/jetson/dofbot_ws/src/dofbot_color_sorting/HSV_config.txt"
# 读取HSV配置文件,更新HSV值  Read HSV configuration file and update HSV value
try: read_HSV(HSV_path,color_hsv)
except Exception: print("Read HSV_config Error!!!")

### 创建控件 Creating widget

In [ ]:
# 创建控件布局  Create widget layout
button_layout  = widgets.Layout(width='200px', height='70px', align_self='center')
# 输出打印  Output printing
output = widgets.Output()
# 退出按钮  exit button
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
# 图像控件  Image widget
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
# 垂直放置  Vertical placement
controls_box = widgets.VBox([imgbox, exit_button], layout=widgets.Layout(align_self='center'))
# ['auto', 'flex-start', 'flex-end', 'center', 'baseline', 'stretch', 'inherit', 'initial', 'unset']

### 控制按钮  Control button

In [ ]:
def exit_button_Callback(value):
    global model
    model = 'Exit'
#     with output: print(model)
exit_button.on_click(exit_button_Callback)

### 主程序 Main process

In [ ]:
def camera():
    # 打开摄像头 Open camera
    capture = cv.VideoCapture(0)
    while capture.isOpened():
        try:
            _, img = capture.read()
            img = cv.resize(img, (640, 480))
#             cv.line(img, (320, 0), (320, 480), color=(0, 255, 0), thickness=1)
#             cv.line(img, (0, 240), (640, 240), color=(0, 255, 0), thickness=1)
            # 获得运动信息  Get motion information
            img = sorting.Sorting_grap(img, color_hsv)
            if model == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except KeyboardInterrupt:capture.release()

### 启动  Start

In [ ]:
display(controls_box,output)
threading.Thread(target=camera, ).start()